# Building with LangChain

In [2]:
import os
import sys
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

load_dotenv()  # Load API keys and other sensitive data from the .env file

True

In [3]:
api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

# LLM Chain

In [4]:
llm = ChatOpenAI(openai_api_key=api_key)  # Create the ChatGPT instance

In [5]:
llm.invoke("where was born the current president of Colombia?")

AIMessage(content="I'm sorry, I do not have information on the current president of Colombia as my training data only goes up until September 2021.")

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class news anchor."),
    ("user", "{input}")
])

In [7]:
chain = prompt | llm 

In [8]:
chain.invoke({"input": "where was born the current president of Colombia?"})

AIMessage(content="I'm sorry, I don't have real-time information. Would you like me to look up the current president of Colombia for you?")

In [9]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [10]:
chain = prompt | llm | output_parser

In [11]:
chain.invoke({"input": "where was born the current president of Colombia?"})

'The current President of Colombia is Iván Duque Márquez. He was born on August 1, 1976 in Bogotá, Colombia.'

# Retrieval Chain

In [12]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://en.wikipedia.org/wiki/President_of_Colombia")

docs = loader.load()

In [13]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=api_key)

In [14]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [15]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [16]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "where was born the current president of Colombia?",
    "context": [Document(page_content="the president of Colombia is Rodolfo Hernández, born in New York")]
})

'The current president of Colombia was born in New York.'

In [17]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [18]:
response = retrieval_chain.invoke({"input": "where was born the current president of Colombia?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

The provided context does not mention where the current president of Colombia was born.


# Conversation Retrieval Chain

In [19]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [20]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can you tell me about the current president of Colombia?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "where was he born"
})

[Document(page_content='Incumbency[edit]\nResidence[edit]\nCasa de Nariño\nCurrently the official residence of the Colombian president is the Casa de Nariño.[19] While in office, the president has the right to use its facilities and personnel, including medical care, recreation, domestic services, and security services. The first presidential palace where Simón Bolívar dispatched after Independence of Colombia was the former Palace of the Viceroys, which was located on the western side of the Plaza Mayor of Bogotá (where the Palacio Liévano is currently located).[20] The 1827 earthquake left the property partially destroyed, for which reason the presidential office and the official residence were moved to the Palacio de San Carlos.[21] In 1885, the president Rafael Núñez ordered the purchase of the house where Antonio Nariño was born to use it as an official residence.[22] In 1906, the architect Gastón Lelarge transformed the property into the Palacio de la Carrera and in 1979 the arch

In [21]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [22]:
chat_history = [HumanMessage(content="Can you tell me about the president of Colombia?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "where was he born"
})

{'chat_history': [HumanMessage(content='Can you tell me about the president of Colombia?'),
  AIMessage(content='Yes!')],
 'input': 'where was he born',
 'context': [Document(page_content='Incumbency[edit]\nResidence[edit]\nCasa de Nariño\nCurrently the official residence of the Colombian president is the Casa de Nariño.[19] While in office, the president has the right to use its facilities and personnel, including medical care, recreation, domestic services, and security services. The first presidential palace where Simón Bolívar dispatched after Independence of Colombia was the former Palace of the Viceroys, which was located on the western side of the Plaza Mayor of Bogotá (where the Palacio Liévano is currently located).[20] The 1827 earthquake left the property partially destroyed, for which reason the presidential office and the official residence were moved to the Palacio de San Carlos.[21] In 1885, the president Rafael Núñez ordered the purchase of the house where Antonio Nariñ

# Agent

We've so far create examples of chains - where each step is known ahead of time. The final thing we will create is an agent - where the LLM decides what steps to take.

NOTE: for this example we will only show how to create an agent using OpenAI models, as local models are not reliable enough yet.

One of the first things to do when building an agent is to decide what tools it should have access to. For this example, we will give the agent access to two tools:

The retriever we just created. This will let it easily answer questions about LangSmith
A search tool. This will let it easily answer questions that require up to date information.
First, let's set up a tool for the retriever we just created:

In [23]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about the president of Colombia. For any questions about him, you must use this tool!",
)

In [24]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [25]:
tools = [retriever_tool, search]

In [26]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [27]:
agent_executor.invoke({"input": "who is the president of Colombia?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'current president of Colombia'}`


President of Colombia - Wikipedia




































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload file



















Search











Search





























Create account

Log in








Personal tools





 Create account Log in





		Pages for logged out editors learn more



ContributionsTalk




























Contents
move to sidebar
hide




(Top)





1Executive powers



Toggle Executive powers subsection





1.1Commander-in-chief







1.2Administrative powers







1.3Judicial powers







1.4Foreign affairs









2Leadership roles







3Selection process



Toggle Selection process subsection





3.1Eligib

{'input': 'who is the president of Colombia?',
 'output': 'The current President of Colombia is Gustavo Petro, who assumed office on August 7, 2022.'}

In [ ]:
agent_executor.invoke({"input": "where was he born?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'President of Colombia'}`


President of Colombia - Wikipedia




































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload file



















Search











Search





























Create account

Log in








Personal tools





 Create account Log in





		Pages for logged out editors learn more



ContributionsTalk




























Contents
move to sidebar
hide




(Top)





1Executive powers



Toggle Executive powers subsection





1.1Commander-in-chief







1.2Administrative powers







1.3Judicial powers







1.4Foreign affairs









2Leadership roles







3Selection process



Toggle Selection process subsection





3.1Eligibility




In [ ]:
chat_history = [HumanMessage(content="Is he a man?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "how old is he?"
})

# Serving with LangServe
Now that we've built an application, we need to serve it. That's where LangServe comes in. LangServe helps developers deploy LangChain chains as a REST API. You do not need to use LangServe to use LangChain, but in this guide we'll show how you can deploy your app with LangServe.

While the first part of this guide was intended to be run in a Jupyter Notebook, we will now move out of that. We will be creating a Python file and then interacting with it from the command line.

Install with:

In [33]:
#!/usr/bin/env python
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage
from langserve import add_routes

# 1. Load Retriever
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

# 2. Create Tools
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
search = TavilySearchResults()
tools = [retriever_tool, search]


# 3. Create Agent
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

# We need to add these input/output schemas because the current AgentExecutor
# is lacking in schemas.

class Input(BaseModel):
    input: str
    chat_history: List[BaseMessage] = Field(
        ...,
        extra={"widget": {"type": "chat", "input": "location"}},
    )


class Output(BaseModel):
    output: str

add_routes(
    app,
    agent_executor.with_types(input_type=Input, output_type=Output),
    path="/agent",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop

we should see our chain being served at localhost:8000.

# Playground
Every LangServe service comes with a simple built-in UI for configuring and invoking the application with 
streaming output and visibility into intermediate steps. Head to http://localhost:8000/agent/playground/ 
to try it out! Pass in the same question as before - "how can langsmith help with testing?" - and it 
should respond same as before.

# Client
Now let's set up a client for programmatically interacting with our service. We can easily do this 
with the [langserve.RemoteRunnable](/docs/langserve#client). Using this, we can interact with the 
served chain as if it were running client-side.